# 📊 EDA Provvisoria - Creazione Burnout Labels

## Scopo

Questo notebook è una versione preliminare/provvisoria dell'EDA focalizzata sulla **creazione del target variable** (burnout_score e burnout_level).

### Workflow
1. Caricamento dei 4 dataset raw da Kaggle
2. Creazione del burnout score composito (z-score)
3. Discretizzazione in 3 classi usando percentili
4. Merge con i dati giornalieri per propagare il label

> **Nota**: La versione finale e più completa è in `01_eda.ipynb`

In [ ]:
# =============================================================================
# CARICAMENTO DATI
# =============================================================================
# Importiamo le librerie necessarie e carichiamo tutti i dataset
# dal folder data/raw/ (scaricati da Kaggle)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

# Caricamento dei 4 file del dataset
daily_logs = pd.read_csv("../data/raw/daily_logs.csv")      # Metriche giornaliere
daily_all = pd.read_csv("../data/raw/daily_all.csv")        # Versione espansa
interventions = pd.read_csv("../data/raw/interventions.csv") # Interventi wellness
weekly = pd.read_csv("../data/raw/weekly_summaries.csv")     # Riepiloghi settimanali

# Stampa dimensioni per verifica
print("Daily logs:", daily_logs.shape)      # ~365k righe (1000 users × 365 days)
print("Daily all:", daily_all.shape)
print("Interventions:", interventions.shape)
print("Weekly summaries:", weekly.shape)    # ~52k righe (1000 users × 52 weeks)

Daily logs: (731000, 26)
Daily all: (731000, 53)
Interventions: (332, 6)
Weekly summaries: (105000, 10)


In [ ]:
# =============================================================================
# CREAZIONE DEL BURNOUT SCORE
# =============================================================================
# Il burnout non è direttamente osservabile nel dataset.
# Lo deriviamo combinando indicatori psicometrici correlati.
#
# FORMULA:
# burnout = (z_stress + z_anxiety + z_depression + z_sleep_debt - z_job_satisfaction) / 5
#
# Dove z_* indica il valore standardizzato (z-score) di ciascuna variabile.
# La standardizzazione rende le scale comparabili.

import numpy as np
from sklearn.preprocessing import StandardScaler

# Selezioniamo le colonne che contribuiscono al burnout
# Queste sono disponibili nel dataset weekly_summaries
burnout_features = weekly[[
    "perceived_stress_scale",  # PSS-10: Perceived Stress Scale (0-40)
    "anxiety_score",           # GAD-7: Generalized Anxiety Disorder scale
    "depression_score",        # PHQ-9: Patient Health Questionnaire for depression
    "sleep_debt_hours",        # Ore di sonno perse rispetto al fabbisogno ideale
    "job_satisfaction",        # Scala di soddisfazione lavorativa (1-10)
]]

# Standardizzazione z-score
# Trasforma ogni colonna in: (x - media) / deviazione_standard
# Risultato: media = 0, std = 1 per ogni colonna
scaler = StandardScaler()
burnout_z = scaler.fit_transform(burnout_features)

burnout_z = pd.DataFrame(
    burnout_z,
    columns=burnout_features.columns,
    index=weekly.index
)

# Calcolo del burnout score composito
# NOTA: job_satisfaction ha coefficiente NEGATIVO perché è un fattore protettivo
# (alta soddisfazione = basso burnout)
weekly["burnout_score"] = (
    burnout_z["perceived_stress_scale"]
    + burnout_z["anxiety_score"]
    + burnout_z["depression_score"]
    + burnout_z["sleep_debt_hours"]
    - burnout_z["job_satisfaction"]  # Sottratto!
) / 5.0  # Media dei 5 contributi

In [ ]:
# =============================================================================
# STATISTICHE DESCRITTIVE DEL BURNOUT SCORE
# =============================================================================
# Verifichiamo che il burnout score abbia una distribuzione ragionevole

weekly["burnout_score"].describe()

# Output atteso:
# - mean ≈ 0 (perché usiamo z-scores)
# - std ≈ 0.4-0.6 (combinazione di 5 variabili standardizzate)
# - min/max ragionevoli (niente outlier estremi)

count    1.050000e+05
mean    -1.808162e-16
std      6.748538e-01
min     -2.011962e+00
25%     -4.795661e-01
50%     -1.075208e-01
75%      3.998777e-01
max      4.497371e+00
Name: burnout_score, dtype: float64

In [ ]:
# =============================================================================
# DISCRETIZZAZIONE IN 3 CLASSI
# =============================================================================
# Per la classificazione, convertiamo il burnout continuo in 3 livelli:
# - 0 = Low (basso burnout)
# - 1 = Medium (medio)
# - 2 = High (alto burnout)
#
# Usiamo i percentili 33° e 66° per garantire classi bilanciate.

low_thr = weekly["burnout_score"].quantile(0.33)   # Soglia tra low e medium
high_thr = weekly["burnout_score"].quantile(0.66)  # Soglia tra medium e high

def burnout_class(score):
    """Converte il burnout score continuo in classe discreta."""
    if score < low_thr:
        return 0  # Low burnout
    elif score < high_thr:
        return 1  # Medium burnout
    else:
        return 2  # High burnout

weekly["burnout_level"] = weekly["burnout_score"].apply(burnout_class)

# Verifica bilanciamento classi
# Dovremmo vedere circa 33% per ogni classe
weekly["burnout_level"].value_counts()

burnout_level
2    35700
1    34658
0    34642
Name: count, dtype: int64

In [ ]:
# =============================================================================
# MERGE CON DATI GIORNALIERI
# =============================================================================
# Il burnout è calcolato a livello settimanale, ma vogliamo usare le
# features giornaliere per il training. Facciamo un merge basato su
# (user_id, settimana) per propagare il burnout_level a ogni giorno.

# Convertiamo le date in datetime
daily_logs["date"] = pd.to_datetime(daily_logs["date"])
weekly["week_start"] = pd.to_datetime(weekly["week_start"])

# Estraiamo il numero della settimana ISO (1-52)
daily_logs["week"] = daily_logs["date"].dt.isocalendar().week
weekly["week"] = weekly["week_start"].dt.isocalendar().week

# Left join: manteniamo tutti i record daily
# e aggiungiamo burnout_score/level dalla settimana corrispondente
merged = pd.merge(
    daily_logs,
    weekly[["user_id", "week", "burnout_score", "burnout_level"]],
    on=["user_id", "week"],
    how="left"
)

print(f"Merged dataset shape: {merged.shape}")
print(f"Missing burnout values: {merged['burnout_level'].isna().sum()}")